In [2]:

from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.utils_io import Console_and_file_logger
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
import logging
Console_and_file_logger('parse_xml', logging.INFO)

import os
import re
import numpy as np
from scipy import stats
from collections import OrderedDict
import json
import SimpleITK as sitk
from bs4 import BeautifulSoup
from bs4 import SoupStrainer



# specific imports
import xml.etree.ElementTree as ET

2020-05-05 18:08:26,727 INFO -------------------- Start --------------------
2020-05-05 18:08:26,729 INFO Working directory: /mnt/data/git/cardio.
2020-05-05 18:08:26,730 INFO Log file: ./logs/parse_xml.log


search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


In [15]:
import glob
files = glob.glob('/mnt/data/datasets/cardio/GCN/temp/**/SAX*/*')
len(files)

26

In [16]:
# rename all files
[os.rename('{}'.format(f),'{}.dcm'.format(f)) for f in files]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [64]:
# helper function to describe an xml element

def describe_element(element):
    print('Element-Tag: {}'.format(element.tag))
    print('Element: {}'.format(element))
    
    #returns dict
    print('Element-Atribs: {}'.format(element.attrib))
    
    # returns list
    print('Element items: {}'.format(element.items()))
    print('Len Element Items: {}'.format(len(element.items())))
    print('Key: {}'.format(element.items()[0][1]))
    print('Type: {}'.format(element.items()[1][1]))
    print('Text: {}'.format(element.text))
    print()
    

In [77]:
# every tag is "{http://www.circlecvi.com/cvi42/Workspace/Hash/}item"
# returns a list of all sop instance uids for an xml file-path
def get_uids_from_segmentation_file(xml_file):
    sop_instance_uids = []
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for elem in root.iter('{http://www.circlecvi.com/cvi42/Workspace/Hash/}item'):
        if elem.attrib['{http://www.circlecvi.com/cvi42/Workspace/Hash/}key'] == 'StudyMapStates':
            
            for elem1 in elem[0]: # elem = list with 2 buckets
                describe_element(elem1)
                if elem1.items()[0][1] == 'ImageStates': # access the Key of the attributes
                    #describe_element(elem1)
                    for elem2 in elem1:
                        #print(elem2.items()[0][1])
                        #print(elem2.items())
                        sop_instance_uids.append(elem2.items()[0][1])
                        #describe_element(elem2)
                        
                        
                        
                        """
                        for elem3 in elem2:
                            keys = elem3.items()[0]
                            if keys[1] == 'ViewGeometry':
                                print(keys[1].tag)
                                print(keys[1].attrib)
                        """




                        #describe_element(elem2)

    print(len(sop_instance_uids))
    print(sop_instance_uids[0:10])
    return sop_instance_uids

In [132]:
def has_key(elem, key):
    #describe_element(elem)
    if elem.items()[0][1] == key:
        return True

In [153]:
def get_value(elem):
    return elem.items()[0][1]

In [169]:
def describe(xml_file):
    sop_instance_uids = []
    tree = ET.parse(xml_file)
    root = tree.getroot()
    n_space = '{http://www.circlecvi.com/cvi42/Workspace/Hash/}'

    for elem in root.iter('{http://www.circlecvi.com/cvi42/Workspace/Hash/}item'):
        if elem.attrib['{http://www.circlecvi.com/cvi42/Workspace/Hash/}key'] == 'StudyMapStates':
            
            for elem1 in elem[0]: # elem = list with 2 buckets
                #describe_element(elem1) # elem1 = Manual Series, StudyUID and ImageState
                if has_key(elem1, 'ImageStates'):
                    print('IMAGES GEFUNDEN')
                    print()
                    for idx, image in enumerate(elem1): # iterate over all image elements

                        sop_instance_uid = get_value(image)
                        #describe_element(image)
                        for image_tag in image:
                            if has_key(image_tag, 'Contours'): # get all contour tags
                                if image_tag.attrib[n_space + 'count'] == '1': # if this image has a contour element
                                    print(image_tag.attrib[n_space + 'count'])
                                    #describe_element(image_tag)
                                    #print(sop_instance_uid)
                                    sop_instance_uids.append(sop_instance_uid)
                                     
                                    
                    
    print(sop_instance_uids)
    print(len(sop_instance_uids))
                
                
                

In [172]:
xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'

instance_uids = describe(xml_file)

IMAGES GEFUNDEN

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
['1.3.12.2.1107.5.99.2.1013.30000008030612131778100068112', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068115', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068118', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068121', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068124', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068127', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068031', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068034', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068037', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068040', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068043', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068046', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068049', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068025', '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068028', '1.3.12.2.1107.5.99.2

In [48]:
xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'

with open(xml_file) as fp:
    soup = BeautifulSoup(fp, 'lxml')

In [3]:
# test with beautiful-soup
contours = {'saendocardialContour': [],
           'saepicardialContour' : [],
           'sarvendocardialContour': [],
           'sarvepicardialContour': []}
contour = 'saendocardialContour'
    
all_items = list(soup.find_all('hash:item'))
    

In [18]:
class Contour():
    
    def __init__(self, uid, image_size, pixel_size, points, sub_pixel_res):
        self.uid = uid
        self.image_size = image_size
        self.pixel_size = pixel_size
        self.points = points
        self.sub_pixel_res = sub_pixel_res
        
    uid = 0  
    image_size = (0,0)
    pixel_size = (0,0)
    sub_pixel_res = 0
    points = []
    

In [62]:
def contour_factory(contours):
    """
    Expects a list of segmentation-elements
    Reads/collects all neccessary data for each contour-tag
    Wraps each contour information in a contour-object,
    builds a list of Contours and returns them
    Params: list of contour-xml tags:
    e.g. call:
    xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'
    with open(xml_file) as fp:
    soup = BeautifulSoup(fp, 'lxml')
    saendocardialContour = soup.find_all('hash:item', {'hash:key' : 'saendocardialContour'})
    
    """
    cont_obj_list = []
    print(len(contours))
    for contour in contours:
        #print(contour)
        uid = contour.parent.parent.attrs.get('hash:key')
        image_width = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:width').text
        image_height = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:height').text
        pixel_width = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:width').text
        pixel_height = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:height').text
        points_t = contour.find('hash:item', {'hash:key' : 'Points'})
        points_x = [point.text for point in points_t.find_all('point:x')]
        points_y = [point.text for point in points_t.find_all('point:y')]
        points = list(zip(points_x, points_y))
        sub_pixel = contour.find('hash:item', {'hash:key' : 'SubpixelResolution'}).text
        print(uid)
        cont_obj_list.append(Contour(uid, 
                                     (image_width, image_height), 
                                     (pixel_width, pixel_height),
                                    points,
                                    sub_pixel))
    return cont_obj_list
        

In [53]:
def extract_contours(soup):
    """
    Expect the Segmentation-XML-File as Beautifulsup-object
    Returns: a dictionary with segmentation-tag : list of segmentation-tag elements
    """
    contours = {'saendocardialContour': [],
           'saepicardialContour' : [],
           'sarvendocardialContour': [],
           'sarvepicardialContour': []}
    
    # Find root element of each contour
    for key, value in contours.items():
        print('read: {}'.format(key))
        contours[key] = soup.find_all('hash:item', {'hash:key' : key})
        print('found: {} contours'.format(len(contours[key])))
    return contours

        

In [55]:
# contours = dictonary
# 
def convert_to_obj(contours):
    """
    Expects a dictionary object with:
    segmentation-tag : list of segmentation-tag elements
    converts the list of segmentation-elements in the dictionary
    to a contour-object with all neccessary attributes
    """
    for key, value in contours.items():
        if len(value) > 0:
            contours[key] = contour_factory(value)
    return contours

In [63]:
# entry point to read all contours

# load all segmentation tags
contours = extract_contours(soup)
contours = convert_to_obj(contours)




read: saendocardialContour
found: 48 contours
read: saepicardialContour
found: 48 contours
read: sarvendocardialContour
found: 51 contours
read: sarvepicardialContour
found: 0 contours
48
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068643
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068616
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068622
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068655
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068661
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068505
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068538
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068544
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068511
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068517
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068526
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068577
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068583
1.3.12.2.1107.5.99.2.1013.30

In [64]:
for key, value in contours.items():
    if len(value) > 0:
        print('key: {}'.format(key))
        print(vars(value[0]))
import json

with open('segmentation.json', 'w') as fp:
    json.dump(data, fp)

key: saendocardialContour
{'uid': '1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604', 'image_size': ('144', '192'), 'pixel_size': ('1.6666666666667', '1.6666666666667'), 'points': [('275', '321'), ('274', '322'), ('273', '322'), ('272', '322'), ('271', '323'), ('270', '323'), ('269', '324'), ('268', '325'), ('267', '325'), ('266', '326'), ('266', '327'), ('265', '328'), ('264', '329'), ('264', '330'), ('263', '331'), ('263', '332'), ('263', '333'), ('262', '334'), ('262', '335'), ('262', '336'), ('261', '337'), ('261', '338'), ('261', '339'), ('261', '340'), ('260', '341'), ('260', '342'), ('260', '343'), ('260', '344'), ('259', '345'), ('259', '346'), ('259', '347'), ('258', '348'), ('258', '349'), ('258', '350'), ('257', '351'), ('257', '352'), ('256', '353'), ('256', '354'), ('255', '355'), ('255', '356'), ('255', '357'), ('254', '358'), ('254', '359'), ('253', '360'), ('253', '361'), ('252', '362'), ('252', '363'), ('252', '364'), ('251', '365'), ('251', '366'), ('251', '367

In [23]:
saendocardialContour = soup.find_all('hash:item', {'hash:key' : 'saendocardialContour'})

print(len(saendocardialContour))

48


In [5]:
print(saendocardialContour[0].parent.parent.attrs.get('hash:key'))

1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604


In [16]:
#elem = saendocardialContour[0]

contour = soup.find('hash:item', {'hash:key' : 'saendocardialContour'})


In [95]:
#print(image_size)
image_width = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:width').text
image_height = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:height').text
print(image_width)
print(image_height)



144
192


In [89]:
pixel_width = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:width').text
pixel_height = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:height').text
print(pixel_width)
print(pixel_height)

1.6666666666667
1.6666666666667


In [93]:
points_t = contour.find('hash:item', {'hash:key' : 'Points'})
points_x = [point.text for point in points_t.find_all('point:x')]
points_y = [point.text for point in points_t.find_all('point:y')]
points = zip(points_x, points_y)

points

In [91]:
sub_pixel = contour.find('hash:item', {'hash:key' : 'SubpixelResolution'}).text
print(sub_pixel)


4


In [33]:
c = contour_factory(saendocardialContour)

1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068643
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068616
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068622
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068655
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068661
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068505
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068538
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068544
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068511
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068517
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068526
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068577
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068583
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068589
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068550
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068556
1.3.12.2.1107.5.99.2.1013.3000000803061213177810

In [22]:
def create_contour(contour):
    
    print(contour.parent.parent.attrs.get('hash:key'))
    image_width = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:width').text
    image_height = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:height').text
    print(image_width)
    print(image_height)
    pixel_width = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:width').text
    pixel_height = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:height').text
    print(pixel_width)
    print(pixel_height)
    points_t = contour.find('hash:item', {'hash:key' : 'Points'})
    points_x = [point.text for point in points_t.find_all('point:x')]
    points_y = [point.text for point in points_t.find_all('point:y')]
    points = zip(points_x, points_y)
    print(points)
    

In [30]:
create_contour(saendocardialContour[0])

1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604
144
192
1.6666666666667
1.6666666666667


In [31]:
contour_factory(saendocardialContour)

1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068643
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068616
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068622
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068655
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068661
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068505
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068538
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068544
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068511
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068517
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068526
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068577
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068583
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068589
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068550
1.3.12.2.1107.5.99.2.1013.30000008030612131778100068556
1.3.12.2.1107.5.99.2.1013.3000000803061213177810

In [43]:
parent = elem.parent.parent
parent

<hash:item hash:count="4" hash:key="1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604">
<hash:item hash:key="WindowWidth" variant:type="double">1006</hash:item>
<hash:item hash:count="3" hash:key="Contours" variant:type="Workspace::MapHash">
<hash:item hash:count="9" hash:key="sarvendocardialContour">
<hash:item hash:key="ImageSize" variant:type="QSize">
<size:width>144</size:width>
<size:height>192</size:height>
</hash:item>
<hash:item hash:key="Points" list:count="343" variant:type="QPolygon">
<list:item list:index="0">
<point:x>230</point:x>
<point:y>356</point:y>
</list:item>
<list:item list:index="1">
<point:x>229</point:x>
<point:y>355</point:y>
</list:item>
<list:item list:index="2">
<point:x>228</point:x>
<point:y>354</point:y>
</list:item>
<list:item list:index="3">
<point:x>228</point:x>
<point:y>353</point:y>
</list:item>
<list:item list:index="4">
<point:x>227</point:x>
<point:y>352</point:y>
</list:item>
<list:item list:index="5">
<point:x>226</point:x>
<point:y>352<

In [44]:
children = elem.children
children

In [45]:
UID = parent.attrs.get('hash:key')
print(UID)

1.3.12.2.1107.5.99.2.1013.30000008030612131778100068604


In [40]:
for child in elem:
    if child.hash:item.hash.key == 'ImageSize':
            print(child)

SyntaxError: invalid syntax (<ipython-input-40-caa25c44a190>, line 2)

In [46]:
elem.children.select()




<hash:item hash:key="ImageSize" variant:type="QSize">
<size:width>144</size:width>
<size:height>192</size:height>
</hash:item>


<hash:item hash:key="Points" list:count="230" variant:type="QPolygon">
<list:item list:index="0">
<point:x>275</point:x>
<point:y>321</point:y>
</list:item>
<list:item list:index="1">
<point:x>274</point:x>
<point:y>322</point:y>
</list:item>
<list:item list:index="2">
<point:x>273</point:x>
<point:y>322</point:y>
</list:item>
<list:item list:index="3">
<point:x>272</point:x>
<point:y>322</point:y>
</list:item>
<list:item list:index="4">
<point:x>271</point:x>
<point:y>323</point:y>
</list:item>
<list:item list:index="5">
<point:x>270</point:x>
<point:y>323</point:y>
</list:item>
<list:item list:index="6">
<point:x>269</point:x>
<point:y>324</point:y>
</list:item>
<list:item list:index="7">
<point:x>268</point:x>
<point:y>325</point:y>
</list:item>
<list:item list:index="8">
<point:x>267</point:x>
<point:y>325</point:y>
</list:item>
<list:item list:index="9

In [214]:
saepicardialContour = soup.find_all('hash:item', {'hash:key' : 'saepicardialContour'})

print(len(saepicardialContour))

48


In [215]:
sarvendocardialContour = soup.find_all('hash:item', {'hash:key' : 'sarvendocardialContour'})

print(len(sarvendocardialContour))

51


In [217]:
sarvepicardialContour = soup.find_all('hash:item', {'hash:key' : 'sarvepicardialContour'})

print(len(sarvepicardialContour))

0


0


In [171]:
# test to get a list of all valid sop uids for one segmentation file

xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'

instance_uids = get_uids_from_segmentation_file(xml_file)
print(len(instance_uids))

Element-Tag: {http://www.circlecvi.com/cvi42/Workspace/Hash/}item
Element: <Element '{http://www.circlecvi.com/cvi42/Workspace/Hash/}item' at 0x000001C689E475E8>
Element-Atribs: {'{http://www.circlecvi.com/cvi42/Workspace/Hash/}key': 'ManualSeries', '{http://www.circlecvi.com/cvi42/Workspace/Variant/}type': 'QList<QList<QList<QString> > >', '{http://www.circlecvi.com/cvi42/Workspace/List/}count': '0'}
Element items: [('{http://www.circlecvi.com/cvi42/Workspace/Hash/}key', 'ManualSeries'), ('{http://www.circlecvi.com/cvi42/Workspace/Variant/}type', 'QList<QList<QList<QString> > >'), ('{http://www.circlecvi.com/cvi42/Workspace/List/}count', '0')]
Len Element Items: 3
Key: ManualSeries
Type: QList<QList<QList<QString> > >
Text: None

Element-Tag: {http://www.circlecvi.com/cvi42/Workspace/Hash/}item
Element: <Element '{http://www.circlecvi.com/cvi42/Workspace/Hash/}item' at 0x000001C689E47048>
Element-Atribs: {'{http://www.circlecvi.com/cvi42/Workspace/Hash/}key': 'StudyUid', '{http://www.

In [66]:
if elem1[0][1] == 'StudyUid':
    print('Study-ID: {}'.format(elem1.text))
if items[0][1] == 'ImageStates': # for every image-state element
    parent = items[0][1]
    for elem in parent.iter():
        print(elem)

Element-Tag: {http://www.circlecvi.com/cvi42/Workspace/Hash/}item
Element: <Element '{http://www.circlecvi.com/cvi42/Workspace/Hash/}item' at 0x000001C689E62B88>
Element-Atribs: {'{http://www.circlecvi.com/cvi42/Workspace/Hash/}key': 'StudyMapStates', '{http://www.circlecvi.com/cvi42/Workspace/Variant/}type': 'Workspace::MapList', '{http://www.circlecvi.com/cvi42/Workspace/List/}count': '2'}
Element items: [('{http://www.circlecvi.com/cvi42/Workspace/Hash/}key', 'StudyMapStates'), ('{http://www.circlecvi.com/cvi42/Workspace/Variant/}type', 'Workspace::MapList'), ('{http://www.circlecvi.com/cvi42/Workspace/List/}count', '2')]
Len Element Items: 3
Key: StudyMapStates
Type: Workspace::MapList
Text: 
        

Element-Tag: {http://www.circlecvi.com/cvi42/Workspace/Hash/}item
Element: <Element '{http://www.circlecvi.com/cvi42/Workspace/Hash/}item' at 0x000001C689E62868>
Element-Atribs: {'{http://www.circlecvi.com/cvi42/Workspace/Hash/}key': 'ImageStates', '{http://www.circlecvi.com/cvi42/Wo

AttributeError: 'str' object has no attribute 'tag'